In [33]:
import numpy as np
import pandas as pd
data = pd.read_excel("StockDataPython.xlsx", header = None)

data = np.array(data)
data = data.squeeze()

for t in range(T-1):
    data[t] = (data[t+1] - data[t])/data[t]
     
S = 100
T = 120
alpha = 0.5
gamma = 0.8
epsilon = 0.3
k_plus = 0.9
k_minus = -0.9

def choose_action(Q_plus, Q_minus, eps, curr_state):
    q_plus = np.array(Q_plus)
    q_minus = np.array(Q_minus)
    summ = (q_plus[curr_state,:] + q_minus[curr_state,:])/2
    diff = q_plus[curr_state,:]  - q_minus[curr_state, :]
    U = summ + np.random.uniform(-0.5, 0.5)*diff
    
    if np.random.uniform(0,1) < eps:
        action = np.random.randint(0,2)
    else:
        action = np.argmax(U[:])
    return action

# updates the Q-values
def update (curr_action, curr_state, Q, next_action, next_state, r,k,Q_type):
    predict = Q[curr_state, curr_action]
    target = r + gamma*Q[next_state, next_action]
    Q[curr_state, curr_action] = Q[curr_state, curr_action] + alpha*piecewise(target - predict, k, Q_type) 
    
        
# for simplicity, Q_type is a string character indicating which Q we are updating
# Q_type = 'POS' or 'NEG'
def piecewise(TD_error, k, Q_type):
    if Q_type == 'POS':
        if TD_error >= 0:
            return (1+k)*TD_error
        else:
            return (1-k)*TD_error
    elif TD_error >= 0:
        return (1-k)*TD_error
    else:
        return (1+k)*TD_error
    
# step function to take a step in the environment
# 0 means low bet and 1 means high bet as actions
def step(curr_action, curr_state, t, market_change):
    reward = (50*curr_action-25)*market_change[t]
    next_state = curr_state
    return reward, next_state


#---------------Start Training Here------------------

for s in range(S):
    curr_state = 0
    Q_plus = np.zeros((1,2))
    Q_minus = np.zeros((1,2))
    
    curr_action = choose_action(Q_plus, Q_minus, epsilon, curr_state)
    
    for t in range(T-1):
        
        reward, next_state = step(curr_action, curr_state, t, data)
        next_action = choose_action(Q_plus, Q_minus, epsilon, next_state)
        
        update(curr_action, curr_state, Q_plus, next_action, next_state, reward, k_plus, "POS")
        update(curr_action, curr_state, Q_minus, next_action, next_state, reward, k_minus, 'NEG')
        
        # update action, states
        curr_action = next_action
        curr_state  = next_state
print(Q_plus, Q_minus)

[[17.35248035 17.35014149]] [[9.08446365 7.27635759]]
